In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# BigFrames AI (semantic) Operator Tutorial

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/googleapis/python-bigquery-dataframes/blob/main/notebooks/experimental/semantic_operators.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/googleapis/python-bigquery-dataframes/blob/main/notebooks/experimental/semantic_operators.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/bigquery/import?url=https://github.com/googleapis/python-bigquery-dataframes/blob/main/notebooks/experimental/semantic_operators.ipynb">
      <img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTW1gvOovVlbZAIZylUtf5Iu8-693qS1w5NJw&s" alt="BQ logo" width="35">
      Open in BQ Studio
    </a>
  </td>
</table>

This notebook provides a hands-on preview of AI operator APIs powered by the Gemini model.

The notebook is divided into two sections. The first section introduces the API syntax with examples, aiming to familiarize you with how AI operators work. The second section applies AI operators to a large real-world dataset and presents performance statistics.

This work is inspired by [this paper](https://arxiv.org/pdf/2407.11418) and powered by BigQuery ML and Vertex AI.

# Preparation

First, import the BigFrames modules.



In [ ]:
import bigframes
import bigframes.pandas as bpd

Make sure the BigFrames version is at least `1.23.0`

In [ ]:
from packaging.version import Version

assert Version(bigframes.__version__) >= Version("1.23.0")

Turn on the semantic operator experiment. You will see a warning sign saying that these operators are still under experiments. If you don't turn on the experiment before using the operators, you will get `NotImplemenetedError`s.

In [ ]:
bigframes.options.experiments.semantic_operators = True

Specify your GCP project and location.

In [ ]:
bpd.options.bigquery.project = 'YOUR_PROJECT_ID'
bpd.options.bigquery.location = 'US'

**Optional**: turn off the display of progress bar so that only the operation results will be printed out

In [ ]:
# bpd.options.display.progress_bar = None

Create LLM instances. They will be passed in as parameters for each semantic operator.

This tutorial uses the "gemini-1.5-flash-002" model for text generation and "text-embedding-005" for embedding. While these are recommended, you can choose [other Vertex AI LLM models](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models) based on your needs and availability. Ensure you have [sufficient quota](https://cloud.google.com/vertex-ai/generative-ai/docs/quotas) for your chosen models and adjust it if necessary.

In [ ]:
from bigframes.ml import llm
gemini_model = llm.GeminiTextGenerator(model_name="gemini-1.5-flash-001")
text_embedding_model = llm.TextEmbeddingGenerator(model_name="text-embedding-005")

**Note**: semantic operators could be expensive over a large set of data. As a result, our team added this option `bigframes.options.compute.sem_ops_confirmation_threshold` at `version 1.31.0` so that the BigFrames will ask for your confirmation if the amount of data to be processed is too large. If the amount of rows exceeds your threshold, you will see a prompt for your keyboard input -- 'y' to proceed and 'n' to abort. If you abort the operation, no LLM processing will be done.

The default threshold is 0, which means the operators will always ask for confirmations. You are free to adjust the value as needed. You can also set the threshold to `None` to disable this feature.

In [ ]:
if Version(bigframes.__version__) >= Version("1.31.0"):
    bigframes.options.compute.semantic_ops_confirmation_threshold = 1000

If you would like your operations to fail automatically when the data is too large, set `bigframes.options.compute.semantic_ops_threshold_autofail` to `True`:

In [ ]:
# if Version(bigframes.__version__) >= Version("1.31.0"):
#     bigframes.options.compute.semantic_ops_threshold_autofail = True

# API Samples

You will learn about each semantic operator by trying some examples.

## Semantic Filtering

Semantic filtering allows you to filter your dataframe based on the instruction (i.e. prompt) you provided.

First, create a dataframe:

In [ ]:
df = bpd.DataFrame({'country': ['USA', 'Germany', 'Japan'], 'city': ['Seattle', 'Berlin', 'Kyoto']})
df

,country,city
0,USA,Seattle
1,Germany,Berlin
2,Japan,Kyoto


Now, filter this dataframe by keeping only the rows where the value in `city` column is the capital of the value in `country` column. The column references could be "escaped" by using a pair of braces in your instruction. In this example, your instruction should be like this:
```
The {city} is the capital of the {country}.
```

Note that this is not a Python f-string, so you shouldn't prefix your instruction with an `f`.

In [ ]:
df.semantics.filter("The {city} is the capital of the {country}", model=gemini_model)

,country,city
1,Germany,Berlin


The filter operator extracts the information from the referenced column to enrich your instruction with context. The instruction is then sent for the designated model for evaluation. For filtering operations, the LLM is asked to return only `True` and `False` for each row, and the operator removes the rows accordingly.

## Semantic Mapping

Semantic mapping allows to you to combine values from multiple columns into a single output based your instruction.

Here is an example:

In [ ]:
df = bpd.DataFrame({
    "ingredient_1": ["Bun", "Soy Bean", "Sausage"],
    "ingredient_2": ["Beef Patty", "Bittern", "Long Bread"]
    })
df

,ingredient_1,ingredient_2
0,Bun,Beef Patty
1,Soy Bean,Bittern
2,Sausage,Long Bread


Now, you ask LLM what kind of food can be made from the two ingredients in each row. The column reference syntax in your instruction stays the same. In addition, you need to specify the column name by setting the `output_column` parameter to hold the mapping results.

In [ ]:
df.semantics.map("What is the food made from {ingredient_1} and {ingredient_2}? One word only.", output_column="food", model=gemini_model)

,ingredient_1,ingredient_2,food
0,Bun,Beef Patty,Burger
1,Soy Bean,Bittern,Tofu
2,Sausage,Long Bread,Hotdog


## Semantic Joining

Semantic joining can join two dataframes based on the instruction you provided.

First, you prepare two dataframes:

In [ ]:
cities = bpd.DataFrame({'city': ['Seattle', 'Ottawa', 'Berlin', 'Shanghai', 'New Delhi']})
continents = bpd.DataFrame({'continent': ['North America', 'Africa', 'Asia']})

You want to join the `cities` with `continents` to form a new dataframe such that, in each row the city from the `cities` data frame is in the continent from the `continents` dataframe. You could re-use the aforementioned column reference syntax:

In [ ]:
cities.semantics.join(continents, "{city} is in {continent}", model=gemini_model)

,city,continent
0,Seattle,North America
1,Ottawa,North America
2,Shanghai,Asia
3,New Delhi,Asia


!! **Important:** Semantic join can trigger probihitively expensitve operations! This operation first cross joins two dataframes, then invokes semantic filter on each row. That means if you have two dataframes of sizes `M` and `N`, the total amount of queries sent to the LLM is on the scale of `M * N`.

### Self Joins

This self-join example is for demonstrating a special case: what happens when the joining columns exist in both data frames? It turns out that you need to provide extra information in your column references: by attaching "left." and "right." prefixes to your column names.

Create an example data frame:

In [ ]:
animals = bpd.DataFrame({'animal': ['cow', 'cat', 'spider', 'elephant']})

You want to compare the weights of these animals, and output all the pairs where the animal on the left is heavier than the animal on the right. In this case, you use `left.animal` and `right.animal` to differentiate the data sources:

In [ ]:
animals.semantics.join(animals, "{left.animal} generally weighs heavier than {right.animal}", model=gemini_model)

,animal_left,animal_right
0,cow,cat
1,cow,spider
2,cat,spider
3,elephant,cow
4,elephant,cat
5,elephant,spider


## Semantic Aggregation

Semantic aggregation merges all the values in a column into one. At this moment you can only aggregate a single column in each oeprator call.

Here is an example:

In [ ]:
df = bpd.DataFrame({
    "Movies": [
        "Titanic",
        "The Wolf of Wall Street",
        "Killers of the Flower Moon",
        "The Revenant",
        "Inception",
        "Shuttle Island",
        "The Great Gatsby",
    ],
})
df

,Movies
0,Titanic
1,The Wolf of Wall Street
2,Killers of the Flower Moon
3,The Revenant
4,Inception
5,Shuttle Island
6,The Great Gatsby


You ask LLM to find the oldest movie:

In [ ]:
agg_df = df.semantics.agg("Find the oldest movie from {Movies}. Reply with only the movie title", model=gemini_model)
agg_df

0    Titanic 

Name: Movies, dtype: string

Instead of going through each row one by one, this operator first batches rows to get many  aggregation results. It then repeatly batches those results for aggregation, until there is only one value left. You could set the batch size with `max_agg_rows` parameter, which defaults to 10.

## Semantic Top K

Semantic Top K selects the top K values based on your instruction. Here is an example:

In [ ]:
df = bpd.DataFrame({"Animals": ["Corgi", "Orange Cat", "Parrot", "Tarantula"]})

You want to find the top two most popular pets:

In [ ]:
df.semantics.top_k("{Animals} are more popular as pets", model=gemini_model, k=2)

,Animals
0,Corgi
1,Orange Cat


Under the hood, the semantic top K operator performs pair-wise comparisons with LLM. The top K results are returned in the order of their indices instead of their ranks.

## Semantic Search

Semantic search searches the most similar values to your query within a single column. Here is an example:

In [ ]:
df = bpd.DataFrame({"creatures": ["salmon", "sea urchin", "baboons", "frog", "chimpanzee"]})
df

,creatures
0,salmon
1,sea urchin
2,baboons
3,frog
4,chimpanzee


You want to get the top 2 creatures that are most similar to "monkey":

In [ ]:
df.semantics.search("creatures", query="monkey", top_k = 2, model = text_embedding_model, score_column='similarity score')

,creatures,similarity score
2,baboons,0.708434
4,chimpanzee,0.635844


Note that you are using a text embedding model this time. This model generates embedding vectors for both your query as well as the values in the search space. The operator then uses BigQuery's built-in VECTOR_SEARCH function to find the nearest neighbors of your query.

In addition, `score_column` is an optional parameter for storing the distances between the results and your query. If not set, the score column won't be attached to the result.

## Semantic Similarity Join

When you want to perform multiple similarity queries in the same value space, you could use similarity join to simplify your call. For example:

In [ ]:
df1 = bpd.DataFrame({'animal': ['monkey', 'spider', 'salmon', 'giraffe', 'sparrow']})
df2 = bpd.DataFrame({'animal': ['scorpion', 'baboon', 'owl', 'elephant', 'tuna']})

In this example, you want to pick the most related animal from `df2` for each value in `df1`.

In [ ]:
df1.semantics.sim_join(df2, left_on='animal', right_on='animal', top_k=1, model=text_embedding_model, score_column='distance')

,animal,animal_1,distance
0,monkey,baboon,0.620521
1,spider,scorpion,0.728024
2,salmon,tuna,0.782141
3,giraffe,elephant,0.7135
4,sparrow,owl,0.810864


!! **Important** Like semantic join, this operator can also be very expensive. To guard against unexpected processing of large dataset, use the `bigframes.options.compute.sem_ops_confirmation_threshold` option to specify a threshold.

## Semantic Cluster

Semantic Cluster group similar values together. For example:

In [ ]:
df = bpd.DataFrame({'Product': ['Smartphone', 'Laptop', 'Coffee Maker', 'T-shirt', 'Jeans']})

You want to cluster these products into 3 groups:

In [ ]:
df.semantics.cluster_by(column='Product', output_column='Cluster ID', model=text_embedding_model, n_clusters=3)

,Product,Cluster ID
0,Smartphone,1
1,Laptop,1
2,Coffee Maker,1
3,T-shirt,1
4,Jeans,1


This operator uses the the embedding model to generate vectors for each value, and then the KMeans algorithm for clustering.

# Performance Analyses

In this section, you will use BigQuery's public data of hacker news to perform some heavy work. We recommend you to check the code without executing them in order to save your time and money. The execution results are attached after each cell for your reference.

First, load 3k rows from the table:

In [ ]:
hacker_news = bpd.read_gbq("bigquery-public-data.hacker_news.full")[['title', 'text', 'by', 'score', 'timestamp', 'type']].head(3000)
hacker_news

,title,text,by,score,timestamp,type
0,<NA>,"Well, most people aren&#x27;t alcoholics, so I...",slipframe,<NA>,2021-06-26 02:37:56+00:00,comment
1,<NA>,"No, you don&#x27;t really <i>need</i> a smartp...",vetinari,<NA>,2023-04-19 15:56:34+00:00,comment
2,<NA>,It&#x27;s for the late Paul Allen RIP. Should&...,lsr_ssri,<NA>,2018-10-16 01:07:55+00:00,comment
3,<NA>,Yup they are dangerous. Be careful Donald Trump.,Sven7,<NA>,2015-08-10 16:05:54+00:00,comment
4,<NA>,"Sure, it&#x27;s totally reasonable. Just point...",nicoburns,<NA>,2020-10-05 11:20:51+00:00,comment
5,<NA>,I wonder how long before special forces start ...,autisticcurio,<NA>,2020-09-01 15:38:50+00:00,comment
6,The Impending NY Tech Apocalypse: Here's What ...,<NA>,gaoprea,3,2011-09-27 22:43:27+00:00,story
7,<NA>,Where would you relocate to? I'm assuming that...,pavel_lishin,<NA>,2011-09-16 19:02:01+00:00,comment
8,Eureca beta is live. A place for your business...,<NA>,ricardos,1,2012-10-15 13:09:32+00:00,story
9,<NA>,"It doesn’t work on Safari, and WebKit based br...",archiewood,<NA>,2023-04-21 16:45:13+00:00,comment


Then, keep only the rows that have text content:

In [ ]:
hacker_news_with_texts = hacker_news[hacker_news['text'].isnull() == False]
len(hacker_news_with_texts)

2556

You can get an idea of the input token length by calculating the average string length.

In [ ]:
hacker_news_with_texts['text'].str.len().mean()

390.05125195618155

**Optional**: You can raise the confirmation threshold for a smoother experience.

In [ ]:
if Version(bigframes.__version__) >= Version("1.31.0"):
    bigframes.options.compute.semantic_ops_confirmation_threshold = 5000

Now it's LLM's turn. You want to keep only the rows whose texts are talking about iPhone. This will take several minutes to finish.

In [ ]:
iphone_comments = hacker_news_with_texts.semantics.filter("The {text} is mainly focused on iPhone", gemini_model)
iphone_comments

,title,text,by,score,timestamp,type
9,<NA>,"It doesn’t work on Safari, and WebKit based br...",archiewood,<NA>,2023-04-21 16:45:13+00:00,comment
420,<NA>,Well last time I got angry down votes for sayi...,drieddust,<NA>,2021-01-11 19:27:27+00:00,comment
815,<NA>,New iPhone should be announced on September. L...,meerita,<NA>,2019-07-30 20:54:42+00:00,comment
1516,<NA>,Why would this take a week? i(phone)OS was ori...,TheOtherHobbes,<NA>,2021-06-08 09:25:24+00:00,comment
1563,<NA>,&gt;or because Apple drama brings many clicks?...,weberer,<NA>,2022-09-05 13:16:02+00:00,comment


The performance of the semantic operators depends on the length of your input as well as your quota. Here are our benchmarks for running the previous operation over data of different sizes. Here are the estimates supposing your quota is [the default 200 requests per minute](https://cloud.google.com/vertex-ai/generative-ai/docs/quotas):

* 800 Rows -> ~4m
* 2550 Rows -> ~13m
* 8500 Rows -> ~40m

These numbers can give you a general idea of how fast the operators run.

Now, use LLM to summarize the sentiments towards iPhone:

In [ ]:
iphone_comments.semantics.map("Summarize the sentiment of the {text}. Your answer should have at most 3 words", output_column="sentiment", model=gemini_model)

,title,text,by,score,timestamp,type,sentiment
9,<NA>,"It doesn’t work on Safari, and WebKit based br...",archiewood,<NA>,2023-04-21 16:45:13+00:00,comment,"Frustrated, but hopeful."
420,<NA>,Well last time I got angry down votes for sayi...,drieddust,<NA>,2021-01-11 19:27:27+00:00,comment,Frustrated and angry.
815,<NA>,New iPhone should be announced on September. L...,meerita,<NA>,2019-07-30 20:54:42+00:00,comment,Excited anticipation.
1516,<NA>,Why would this take a week? i(phone)OS was ori...,TheOtherHobbes,<NA>,2021-06-08 09:25:24+00:00,comment,"Frustrated, critical, obvious."
1563,<NA>,&gt;or because Apple drama brings many clicks?...,weberer,<NA>,2022-09-05 13:16:02+00:00,comment,"Negative, clickbait, Apple."


Here is another example: count the number of rows whose authors have animals in their names.

In [ ]:
hacker_news = bpd.read_gbq("bigquery-public-data.hacker_news.full")[['title', 'text', 'by', 'score', 'timestamp', 'type']].head(3000)
hacker_news

,title,text,by,score,timestamp,type
0,<NA>,"Well, most people aren&#x27;t alcoholics, so I...",slipframe,<NA>,2021-06-26 02:37:56+00:00,comment
1,<NA>,"No, you don&#x27;t really <i>need</i> a smartp...",vetinari,<NA>,2023-04-19 15:56:34+00:00,comment
2,<NA>,It&#x27;s for the late Paul Allen RIP. Should&...,lsr_ssri,<NA>,2018-10-16 01:07:55+00:00,comment
3,<NA>,Yup they are dangerous. Be careful Donald Trump.,Sven7,<NA>,2015-08-10 16:05:54+00:00,comment
4,<NA>,"Sure, it&#x27;s totally reasonable. Just point...",nicoburns,<NA>,2020-10-05 11:20:51+00:00,comment
5,<NA>,I wonder how long before special forces start ...,autisticcurio,<NA>,2020-09-01 15:38:50+00:00,comment
6,The Impending NY Tech Apocalypse: Here's What ...,<NA>,gaoprea,3,2011-09-27 22:43:27+00:00,story
7,<NA>,Where would you relocate to? I'm assuming that...,pavel_lishin,<NA>,2011-09-16 19:02:01+00:00,comment
8,Eureca beta is live. A place for your business...,<NA>,ricardos,1,2012-10-15 13:09:32+00:00,story
9,<NA>,"It doesn’t work on Safari, and WebKit based br...",archiewood,<NA>,2023-04-21 16:45:13+00:00,comment


In [ ]:
hacker_news.semantics.filter("{by} contains animal name", model=gemini_model)

,title,text,by,score,timestamp,type
24,Working Best at Coffee Shops,<NA>,GiraffeNecktie,249,2011-04-19 14:25:17+00:00,story
98,<NA>,i resisted switching to chrome for months beca...,catshirt,<NA>,2011-04-06 08:02:24+00:00,comment
137,FDA reverses marketing ban on Juul e-cigarettes,<NA>,anigbrowl,2,2024-06-06 16:42:40+00:00,story
188,<NA>,I think it&#x27;s more than hazing. It may be ...,bayesianhorse,<NA>,2015-06-18 16:42:53+00:00,comment
209,<NA>,I like the idea of moving that arrow the way h...,rattray,<NA>,2015-06-08 02:15:30+00:00,comment
228,<NA>,I don&#x27;t understand why a beginner would s...,wolco,<NA>,2019-02-03 14:35:43+00:00,comment
290,<NA>,I leaerned more with one minute of this than a...,agumonkey,<NA>,2016-07-16 06:19:39+00:00,comment
303,<NA>,I've suggested a <i>rationale</i> for the tabo...,mechanical_fish,<NA>,2008-12-17 04:42:02+00:00,comment
312,<NA>,Do you have any reference for this?<p>I&#x27;m...,banashark,<NA>,2023-11-13 19:57:00+00:00,comment
322,<NA>,Default search scope is an option in the Finde...,kitsunesoba,<NA>,2017-08-13 17:15:19+00:00,comment


Here are the runtime numbers with 500 requests per minute [raised quota](https://cloud.google.com/vertex-ai/generative-ai/docs/quotas):
* 3000 rows -> ~6m
* 10000 rows -> ~26m